In [1]:
from helper_functions import *
import plotly.graph_objects as go
import pandas as pd
import wandb

In [2]:

from datetime import datetime
now = datetime.now()
datetime_str = now.strftime("%m/%d_%H:%M:%S")
datetime_str

'01/17_21:13:57'

In [4]:

# data_df = pd.read_pickle("Kris_updated_yolov8.pkl")
data_df = pd.read_pickle("Kris_merged_yolov8.pkl")
data_df = preprocess_data(data_df)

In [5]:
# Load current experiment number
try:
    with open('experiment_counter.txt', 'r') as f:
        experiment_num = int(f.read())
except:
    experiment_num = 1

# Increment and save the updated experiment number
with open('experiment_7_counter.txt', 'w') as f:
    f.write(str(experiment_num + 1))

In [6]:
# # helper functions:

# def split_data(df, train_ratio=0.6, valid_ratio=0.2):
#     """
#     Splits a dataframe into train, validate, and test sets.
#     """
    
#     df = df[['car_count', 'month_no', 'day', 'day_of_week', 'hour_of_day', 'min_of_day', 'centered_car_count','combined']]

#     # Calculating the split indices
#     train_idx = int(train_ratio * len(df))
#     valid_idx = train_idx + int(valid_ratio * len(df))

#     # Splitting the original dataframe
#     train = df.iloc[:train_idx]
#     valid = df.iloc[train_idx:valid_idx]
#     test = df.iloc[valid_idx:]
    
#     return train, valid, test

# def expanding_window(train, valid, features, target, gpr, window_size):
#     """
#     Uses an expanding window approach for training and validating the GPR model.
#     """
#     metrics_list = []

#     # Define the initial window
#     start, end = 0, window_size

#     while end <= len(train):
#         # Split the current window data
#         X_train, y_train = train[features][start:end], train[target][start:end]

#         # Train the GPR
#         gpr = train_gpr(gpr, X_train, y_train)

#         # Predict on the validation set
#         y_pred = gpr.predict(valid[features])

#         # Calculate and store the metrics
#         metrics = calculate_metrics(valid[target], y_pred)
#         metrics_list.append(metrics)

#         # Expand the window for the next iteration
#         end += 1

#     return metrics_list

# def aggregate_data(df, agg_column, agg_func='mean'):
#     """
#     Aggregates the data based on the given column and aggregation function.
#     """
#     agg_df = df.groupby(agg_column).agg(agg_func).reset_index()
#     return agg_df

# def split_target_feature(df, feature, target):
    
#     X = df[feature].values.reshape(-1,1)
#     y = df[target].values
    
#     return X, y

# def gpr_train_1(gpr, x_train, y_train):
#     """
#     Trains a Gaussian Process Regressor on the given training data.
#     """
#     # # Prior samples visualization
#     # img_prior_train = plot_gpr_samples_plotly(gpr, n_samples, x_train, y_train)
#     # img_prior_train.show()
#     # wandb.log({'prior_samples': img_prior_train})
    
#     # Save initial kernel parameters as artifacts
#     initial_kernel = str(gpr)
#     with open("initial_kernel.txt", "w") as f:
#         f.write(initial_kernel)
    
#     # Fit the GPR model to the training data
#     gpr.fit(x_train, y_train)
    
#     # # Posterior samples visualization
#     # img_posterior_train = plot_gpr_samples_plotly(gpr, n_samples, x_train, y_train)
#     # wandb.log({'posterior_samples': img_posterior_train})
    
#     # Kernel parameters after fit
#     after_fit_kernel = str(gpr.kernel_)
#     with open("after_fit_kernel.txt", "w") as f:
#         f.write(after_fit_kernel)
    
#     # Log artifacts
#     wandb.save("initial_kernel.txt")
#     wandb.save("after_fit_kernel.txt")
    
#     return gpr

# def gpr_pred(run,step,gpr, x_train, y_train, x_test=None, y_test=None, ):
#     # Predict on training data
#     y_train_pred, y_std = gpr.predict(x_train, return_std=True)

#     # Metrics on training data
#     mae_train = mean_absolute_error(y_train, y_train_pred)
#     mse_train = mean_squared_error(y_train, y_train_pred)
#     rmse_train = np.sqrt(mse_train)
#     r2_train = r2_score(y_train, y_train_pred)
#     mape_train = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100
#     lml_train = gpr.log_marginal_likelihood(gpr.kernel_.theta)

#     metrics_train = {
#         "MAE_train": mae_train,
#         "MSE_train": mse_train,
#         "RMSE_train": rmse_train,
#         "R2_train": r2_train,
#         "MAPE_train": mape_train,
#         "LML_train": lml_train,
#         "step":step
#     }
    
#     # Create a wandb Table for training metrics
#     table_train = wandb.Table(columns=["Metric", "Value"])
#     for key, value in metrics_train.items():
#         table_train.add_data(key, value)
#         run.config[key] = value
        
#     wandb.log({"Metrics Table Train": table_train,"step":step})
#     wandb.log(metrics_train)

#     if x_test is not None and y_test is not None:
#         # Predict on test/validation data
#         y_pred, y_std = gpr.predict(x_test, return_std=True)

#         # Metrics on test/validation data
#         mae_test = mean_absolute_error(y_test, y_pred)
#         mse_test = mean_squared_error(y_test, y_pred)
#         rmse_test = np.sqrt(mse_test)
#         r2_test = r2_score(y_test, y_pred)
#         mape_test = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
#         lml_test = gpr.log_marginal_likelihood(gpr.kernel_.theta)

#         metrics_test = {
#             "MAE_test": mae_test,
#             "MSE_test": mse_test,
#             "RMSE_test": rmse_test,
#             "R2_test": r2_test,
#             "MAPE_test": mape_test,
#             "LML_test": lml_test,
#             "step":step
#         }
        
    
#         # Create a wandb Table for test/validation metrics
#         table_test = wandb.Table(columns=["Metric", "Value"])
#         for key, value in metrics_test.items():
#             table_test.add_data(key, value)
#             run.config[key] = value
#         wandb.log({"Metrics Table Test": table_test, "step":step})
#         wandb.log(metrics_test)
#     else:
#         y_pred, y_std = None, None

#     # Log the kernel parameters
#     kernel_str = str(gpr.kernel_)
#     with open("kernel.txt", "w") as f:
#         f.write(kernel_str)
#     wandb.save("kernel.txt")

#     return y_pred, y_std, gpr

# def plot_gpr_posterior_plotly(gpr, X_test,y_test, y_pred, y_std):
#     '''
#     Plots the posterior distribution of a GPR model using Plotly.
#     X_test here is just used to create the x axis margins of plot
#     y_pred can be any predictions from test or validate set
#     '''
#     x = np.linspace(X_test.min(), X_test.max(), len(X_test))
    
#     #pulling samples from test set on a trained gpr
#     y_samples = gpr.sample_y(X_test, 5)

#     fig = go.Figure()


#     for idx, single_prior in enumerate(y_samples.T):
#         fig.add_trace(
#             go.Scatter(
#                 x=x, y=single_prior,
#                 mode="lines",
#                 name=f"Sampled fxn #{idx + 1}",
#                 line=dict(dash="dash")
#             )
#         )

#     fig.add_trace(
#         go.Scatter(
#             x=x,
#             y=y_pred,
#             mode="lines",
#             line_color="red",
#             name="Mean",
            
#         )
#     )

#     fig.add_trace(
#         go.Scatter(
#             x=x,
#             y=y_pred - y_std,
#             fill=None,
#             mode="lines",
#             line_color="rgba(255,0,0,0.1)",
#             showlegend=False,
#         )
#     )

#     fig.add_trace(
#         go.Scatter(
#             x=x,
#             y=y_pred + y_std,
#             fill="tonexty",
#             mode="lines",
#             line_color="rgba(173, 216, 230, 0.5)",
#             name=r"uncertainty",
#         )
#     )


#     fig.add_trace(
#         go.Scatter(
#             x=X_test.ravel(),
#             y=y_test.ravel(),
#             mode='lines+markers',
#             name="Test Data",
#             marker=dict(symbol='cross', size=4, color= 'green')
#         )
#     )

#     fig.update_layout(
#         xaxis_title="Aggregated feature",
#         yaxis_title="Count",
#         height= 800,
#     )

#     # 
#     return  fig


<h2> Exp 7: Traditional tvt split and training

In [6]:
# define splits
train, valid, test = split_data(data_df, train_ratio= 0.6, valid_ratio= 0.2)

# aggregate
train_agg = aggregate_data(train, agg_column='hour_of_day', agg_func='mean')
validate_agg = aggregate_data(valid, agg_column='hour_of_day', agg_func='mean')
test_agg = aggregate_data(test, agg_column='hour_of_day', agg_func='mean')

# separate feature and target

X_train , y_train = split_target_feature(train_agg, feature='hour_of_day', target='centered_car_count')
X_validate , y_validate = split_target_feature(validate_agg, feature='hour_of_day', target='centered_car_count')
X_test , y_test = split_target_feature(test_agg, feature='hour_of_day', target='centered_car_count')


In [7]:
# viz of splits

fig = go.Figure()

fig.add_trace(go.Scatter(x= train_agg['hour_of_day'],
                         y=train_agg['centered_car_count'],
                         mode='markers+lines',
                         name="Training "))

fig.add_trace(go.Scatter(x=validate_agg['hour_of_day'],
                         y=validate_agg['centered_car_count'],
                         mode='markers+lines',
                         name='Validate'))

fig.add_trace(go.Scatter(x=test_agg['hour_of_day'],
                         y=test_agg['centered_car_count'],
                         mode='markers+lines',
                         name='Test'))

fig.show()

In [ ]:
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, ConstantKernel as C
from sklearn.gaussian_process import GaussianProcessRegressor
import wandb

# Define the base kernels
rbf_noise_periodicity= 1 * RBF(length_scale=0.5, length_scale_bounds=(1e-3, 1e3))
locally_periodic = WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
local_global_periodicity = ExpSineSquared(length_scale=1.0, length_scale_bounds=(1e-1, 50), periodicity=1, periodicity_bounds=(1e-1, 50))
k4 = k1 * k3

# List of kernels to test
kernels = [k1 + k2 + k3, k2 + k4, k2 * k4, k4 + k3 + k1, (k4 + k2) + k3, k3]

In [8]:


# Run experiments for each kernel
for i, kernel in enumerate(kernels):
    gpr = GaussianProcessRegressor(kernel=kernel)

    # Initialize wandb run
    datetime_str = now.strftime("%m/%d_%H:%M:%S")
    run = wandb.init(
        project="Single_window_GPR",
        name=f"Experiment_{i+1}_kernel_{kernel}"
    )

    # Configuration parameters
    config = wandb.config
    config.ini_train_win_len = len(train)
    config.valid_win_len = len(valid)
    config.step_size = 1  # 24 hours or 1 day

    # Call the expanding window function
    y_test_pred, y_test_std, gpr = single_window_gpr_process(
        run,
        train,
        valid,
        test,
        gpr,
        features='hour_of_day',
        target='centered_car_count',
        agg_func='mean'
    )

    # Log additional information if necessary
    # wandb.log({})

    # Finish the wandb run
    run.finish()

# End of experiments
print('All experiments completed.')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/sofia_afn/.netrc


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,▁▁
LML_validate,▁
MAE_test,▁
MAE_train,▁▁
MAE_validate,▁
MAPE_test,▁
MAPE_train,▁▁
MAPE_validate,▁
MSE_test,▁
MSE_train,▁▁


wandb: Currently logged in as: afn-sy. Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,▁▁
LML_validate,▁
MAE_test,▁
MAE_train,▁▁
MAE_validate,▁
MAPE_test,▁
MAPE_train,▁▁
MAPE_validate,▁
MSE_test,▁
MSE_train,▁▁


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,▁▁
LML_validate,▁
MAE_test,▁
MAE_train,▁▁
MAE_validate,▁
MAPE_test,▁
MAPE_train,▁▁
MAPE_validate,▁
MSE_test,▁
MSE_train,▁▁


All experiments completed.


In [9]:
# # kernel definition

# from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, ConstantKernel as C

# # RBF kernel with a length scale of 1.0
# k1 = 1 * RBF(length_scale=0.5, length_scale_bounds=(1e-3, 1e3))

# # White kernel to account for noise
# k2 = WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

# # Periodic kernel
# k3 = ExpSineSquared(length_scale=1.0,
#                     length_scale_bounds=(1e-1, 50) ,
#                     periodicity=1,
#                     periodicity_bounds= (1e-1, 50))
# # locally periodic
# k4 = k1 * k3
# # Combined kernel
# # kernel = k1+k2+k3 # this is not really needed
# # kernel = k2 +k4 
# # kernel = k2*k4
# # kernel = k4 + k3 +k1
# # kernel = (k4 + k2)+k3
# kernel= k3

# gpr = GaussianProcessRegressor(kernel=kernel)

In [ ]:
datetime_str = now.strftime("%m/%d_%H:%M:%S")
run = wandb.init(
    # set the wandb project where this run will be logged
    project="Single_window_GPR",
    name= f"Single_window._feature:hour_of_day"
)
# Configuration parameters (this could be extended depending on what you'd like to track)
config = wandb.config
config.ini_train_win_len = len(train)
config.valid_win_len = len(valid)
config.step_size = 1 # 24 hours or 1 day


# Call the expanding window function
y_test_pred, y_test_std, gpr = single_window_gpr_process(run,
                                train,
                                valid,
                                test,
                                gpr,
                                features='hour_of_day',
                                target='centered_car_count',
                                agg_func='mean'
    
)

In [ ]:
# datetime_str = now.strftime("%m/%d_%H:%M:%S")
# run = wandb.init(
#     # set the wandb project where this run will be logged
#     project="Single_window_GPR",
#     name= f"Single_window.{experiment_num}_feature:week"
# )
# # Configuration parameters (this could be extended depending on what you'd like to track)
# config = wandb.config
# config.ini_train_win_len = len(train)
# config.valid_win_len = len(valid)
# config.step_size = 1 # 24 hours or 1 day


# # Call the expanding window function
# y_test_pred, y_test_std, gpr = single_window_gpr_process(run,
#                                 train,
#                                 valid,
#                                 test,
#                                 gpr,
#                                 features='day_of_week',
#                                 target='centered_car_count',
#                                 agg_func='mean'
    
# )

In [ ]:
# datetime_str = now.strftime("%m/%d_%H:%M:%S")
# run = wandb.init(
#     # set the wandb project where this run will be logged
#     project="Single_window_GPR",
#     name= f"Single_window.{experiment_num}_feature:month"
# )
# # Configuration parameters (this could be extended depending on what you'd like to track)
# config = wandb.config
# config.ini_train_win_len = len(train)
# config.valid_win_len = len(valid)
# config.step_size = 1 # 24 hours or 1 day


# # Call the expanding window function
# y_test_pred, y_test_std, gpr = single_window_gpr_process(run,
#                                 train,
#                                 valid,
#                                 test,
#                                 gpr,
#                                 features='day',
#                                 target='centered_car_count',
#                                 agg_func='mean'
    
# )

In [ ]:
k1_fig = plot_gpr_samples_plotly(gpr,5,X_train,y_train)
k2 = 